In [5]:
import pandas as pd
import numpy as np
import json
import calculations
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import statsmodels.api as sm
import warnings
from scipy.stats import iqr, skew, kurtosis, pearsonr, ttest_ind

In [6]:
# Disable runtime warnings due to datasets containing NaNs after skew/kurtosis calcs
warnings.filterwarnings('ignore', category=RuntimeWarning)

# Process data
   
stimuli = []
outlier = []
scale_type = {
    "gesture_pitch_number": 100,
    "gesture_roll_number": 100,
    "slider_number": 100,
    "gesture_pitch_greyscale": 49,
    "gesture_roll_greyscale": 49,
    "slider_greyscale": 49,
}

outlier_cut_off = {
    "gesture_pitch_number": 50,
    "gesture_roll_number": 50,
    "slider_number": 50,
    "gesture_pitch_greyscale": 25,
    "gesture_roll_greyscale": 25,
    "slider_greyscale": 25,
}

truth_value_batches_100 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)],
    [i for i in range(50, 60)],
    [i for i in range(60, 70)],
    [i for i in range(70, 80)],
    [i for i in range(80, 90)],
    [i for i in range(90, 101)]
]

truth_value_batches_49 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)]
]

experiment_truth_type = {
    "gesture_pitch_number": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_pitch_greyscale": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_roll_number": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "gesture_roll_greyscale": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "slider_number": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
    "slider_greyscale": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
}

def ensure_not_outlier(value, experiment_type):
    cut_off = outlier_cut_off[experiment_type]
    return value < cut_off

with open('experiment_data/experiment_data_14.json', 'r') as file:
    data = json.load(file)

    if data:
        for experiment in data["completedExperiments"]:
            experiment_type = experiment.get("experimentType")
            started_date = experiment.get("startedDate")
            ended_date = experiment.get("endedDate")
            for stimulus in experiment["successfulStimuli"]:
                sensor_reading = stimulus.get("sensorReading", {})
                quaternion = (
                    sensor_reading.get("x"),
                    sensor_reading.get("y"),
                    sensor_reading.get("z"),
                    sensor_reading.get("w"),
                )
                _, roll, pitch = calculations.quaternion_to_euler(*quaternion)
                
                sensor_reading["roll"] = roll
                sensor_reading["pitch"] = pitch

                pitch_scaled = calculations.scale_pitch(pitch, scale_type[experiment_type])
                roll_scaled = calculations.scale_roll(roll, scale_type[experiment_type])
                truth_value = stimulus.get("truth")
                slider_value = stimulus.get("value")
                
                pitch_difference = abs(truth_value - pitch_scaled)
                roll_difference = abs(truth_value - roll_scaled)

                stimulus["pitch"] = pitch
                stimulus["pitch_truth"] = pitch_scaled
                stimulus["pitch_truth_diff"] = pitch_difference
            
                stimulus["roll"] = roll
                stimulus["roll_truth"] = roll_scaled
                stimulus["roll_truth_diff"] = roll_difference
                
                stimulus["slider_diff"] = abs(truth_value - slider_value)
                
                if truth_value == 0:
                    stimulus["pitch_truth_diff_percentage"] = (pitch_scaled / scale_type[experiment_type]) * 100
                    stimulus["roll_truth_diff_percentage"] = (roll_scaled / scale_type[experiment_type]) * 100
                else:
                    stimulus["pitch_truth_diff_percentage"] = (pitch_difference / truth_value) * 100
                    stimulus["roll_truth_diff_percentage"] = (roll_difference / truth_value) * 100
                    
                stimulus["experiment_type"] = experiment_type
                stimulus["experiment_time_spent"] = ended_date - started_date
                stimulus["calibration"] = sensor_reading.get("calibration_status")
                
                diff = stimulus[experiment_truth_type[experiment_type]["truth_diff"]]
                if ensure_not_outlier(diff, experiment_type):
                    stimuli.append(stimulus)
                else:
                    outlier.append(stimulus)

df = pd.DataFrame(data=stimuli)
o_df = pd.DataFrame(data=outlier)

In [7]:
print(f'Valid stimuli: {len(df)}')
print(f'Outliers: {len(o_df)}')
print(f'Percentage of outliers: {len(o_df) / len(df) * 100}')

Valid stimuli: 106
Outliers: 14
Percentage of outliers: 13.20754716981132


In [8]:
df[df["experiment_type"] == "gesture_pitch_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
0,"{'w': 0.00860596, 'z': 0.789124, 'stability': ...",device,0.0,20,1,13.462403,7,13,-15.813298,9,11,20.0,65.000000,55.000000,gesture_pitch_greyscale,139.32251,2
1,"{'duration': 167, 'x': -0.204102, 'stability':...",device,0.0,5,2,21.499965,12,7,-13.396093,7,2,5.0,140.000000,40.000000,gesture_pitch_greyscale,139.32251,2
2,"{'stability': 'In motion', 'z': 0.62384, 'cali...",device,0.0,29,4,81.944114,45,16,-2.046509,1,28,29.0,55.172414,96.551724,gesture_pitch_greyscale,139.32251,2
3,"{'calibration_status': 2, 'x': -0.282837, 'y':...",device,0.0,37,8,36.962015,20,17,-13.806352,8,29,37.0,45.945946,78.378378,gesture_pitch_greyscale,139.32251,2
4,"{'timestamp': 1719319653, 'y': -0.483765, 'dur...",device,0.0,43,9,68.391026,37,6,-9.929701,5,38,43.0,13.953488,88.372093,gesture_pitch_greyscale,139.32251,2
5,"{'calibration_status': 2, 'x': -0.081665, 'sta...",device,0.0,22,11,4.023380,2,20,-8.500110,5,17,22.0,90.909091,77.272727,gesture_pitch_greyscale,139.32251,2
6,"{'calibration_status': 2, 'timestamp': 1719319...",device,0.0,38,12,32.401936,18,20,-8.218408,4,34,38.0,52.631579,89.473684,gesture_pitch_greyscale,139.32251,2
7,"{'calibration_status': 2, 'timestamp': 1719319...",device,0.0,30,13,69.000765,38,8,-15.142337,8,22,30.0,26.666667,73.333333,gesture_pitch_greyscale,139.32251,2
8,"{'z': 0.728394, 'activity_confidence': 76, 'du...",device,0.0,26,14,55.030589,30,4,-11.442688,6,20,26.0,15.384615,76.923077,gesture_pitch_greyscale,139.32251,2
9,"{'duration': 157, 'x': -0.345032, 'timestamp':...",device,0.0,12,15,46.557802,25,13,-16.140466,9,3,12.0,108.333333,25.000000,gesture_pitch_greyscale,139.32251,2


In [9]:
df[df["experiment_type"] == "gesture_pitch_number"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
13,"{'w': 0.180237, 'x': -0.390442, 'calibration_s...",device,0.0,64,1,45.932695,51,13,-25.042959,28,36,64.0,20.312500,56.250000,gesture_pitch_number,112.335365,2
14,"{'w': 0.224854, 'x': -0.380798, 'calibration_s...",device,0.0,67,2,49.044438,54,13,-21.739210,24,43,67.0,19.402985,64.179104,gesture_pitch_number,112.335365,2
15,"{'calibration_status': 2, 'duration': 181, 'y'...",device,0.0,91,3,70.892690,79,12,-17.946928,20,71,91.0,13.186813,78.021978,gesture_pitch_number,112.335365,2
16,"{'x': -0.392883, 'activity_confidence': 91, 'z...",device,0.0,62,4,48.791011,54,8,-23.174989,26,36,62.0,12.903226,58.064516,gesture_pitch_number,112.335365,2
17,"{'z': 0.732666, 'timestamp': 1719319747, 'acti...",device,0.0,74,5,59.346314,66,8,-24.082532,27,47,74.0,10.810811,63.513514,gesture_pitch_number,112.335365,2
18,"{'duration': 190, 'activity_confidence': 94, '...",device,0.0,68,6,48.647061,54,14,-29.189574,32,36,68.0,20.588235,52.941176,gesture_pitch_number,112.335365,2
19,"{'calibration_status': 2, 'y': -0.493164, 'tim...",device,0.0,28,7,27.000324,30,2,-30.461561,34,6,28.0,7.142857,21.428571,gesture_pitch_number,112.335365,2
20,"{'timestamp': 1719319768, 'activity_confidence...",device,0.0,56,8,45.326728,50,6,-24.762911,28,28,56.0,10.714286,50.000000,gesture_pitch_number,112.335365,2
21,"{'z': 0.651733, 'calibration_status': 2, 'dura...",device,0.0,57,9,46.729194,52,5,-24.099269,27,30,57.0,8.771930,52.631579,gesture_pitch_number,112.335365,2
22,"{'duration': 154, 'y': -0.540833, 'x': -0.5241...",device,0.0,93,10,70.042286,78,15,-20.931239,23,70,93.0,16.129032,75.268817,gesture_pitch_number,112.335365,2


In [10]:
df[df["experiment_type"] == "gesture_roll_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
33,"{'z': 0.689941, 'x': -0.481995, 'y': -0.472534...",device,0.0,44,1,13.376283,7,37,-65.808874,36,8,44.0,84.090909,18.181818,gesture_roll_greyscale,206.524229,1
34,"{'activity_confidence': 73, 'y': -0.430176, 'c...",device,0.0,23,2,22.423382,12,11,-75.377780,41,18,23.0,47.826087,78.260870,gesture_roll_greyscale,206.524229,1
35,"{'duration': 191, 'w': -0.209595, 'activity_co...",device,0.0,36,3,2.314338,1,35,-48.042855,26,10,36.0,97.222222,27.777778,gesture_roll_greyscale,206.524229,1
36,"{'duration': 154, 'activity_confidence': 83, '...",device,0.0,9,4,2.277466,1,8,-51.946909,28,19,9.0,88.888889,211.111111,gesture_roll_greyscale,206.524229,1
37,"{'duration': 191, 'timestamp': 1719319897, 'w'...",device,0.0,25,5,1.087700,1,24,-28.642069,16,9,25.0,96.000000,36.000000,gesture_roll_greyscale,206.524229,2
38,"{'calibration_status': 1, 'duration': 154, 'st...",device,0.0,32,6,0.670309,0,32,-39.366854,21,11,32.0,100.000000,34.375000,gesture_roll_greyscale,206.524229,1
39,"{'y': 0.219299, 'timestamp': 1719319938, 'stab...",device,0.0,49,7,1.383910,1,48,-62.179796,34,15,49.0,97.959184,30.612245,gesture_roll_greyscale,206.524229,2
40,"{'timestamp': 1719319973, 'calibration_status'...",device,0.0,6,9,-0.864269,0,6,-25.683753,14,8,6.0,100.000000,133.333333,gesture_roll_greyscale,206.524229,2
41,"{'duration': 208, 'w': -0.141602, 'x': -0.2551...",device,0.0,24,10,-1.246253,1,23,-33.909533,18,6,24.0,95.833333,25.000000,gesture_roll_greyscale,206.524229,2
42,"{'z': 0.833679, 'timestamp': 1719319985, 'cali...",device,0.0,15,11,-1.209508,1,14,-42.167901,23,8,15.0,93.333333,53.333333,gesture_roll_greyscale,206.524229,2


In [11]:
df[df["experiment_type"] == "gesture_roll_number"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
51,"{'duration': 213, 'activity_confidence': 95, '...",device,0.0,75,1,-6.540093,7,68,-64.960044,72,3,75.0,90.666667,4.000000,gesture_roll_number,84.541167,2
52,"{'activity_confidence': 96, 'w': -0.284912, 's...",device,0.0,81,2,-6.503005,7,74,-69.505106,77,4,81.0,91.358025,4.938272,gesture_roll_number,84.541167,2
53,"{'z': 0.770508, 'stability': 'Stable', 'y': -0...",device,0.0,70,3,-5.603559,6,64,-59.262947,66,4,70.0,91.428571,5.714286,gesture_roll_number,84.541167,2
54,"{'activity': 'Still', 'calibration_status': 2,...",device,0.0,26,4,-4.109254,5,21,-36.102319,40,14,26.0,80.769231,53.846154,gesture_roll_number,84.541167,2
55,"{'calibration_status': 2, 'activity': 'Still',...",device,0.0,20,5,-3.932020,4,16,-33.662836,37,17,20.0,80.000000,85.000000,gesture_roll_number,84.541167,2
56,"{'calibration_status': 2, 'y': -0.397583, 'tim...",device,0.0,63,6,-6.837365,8,55,-58.008506,64,1,63.0,87.301587,1.587302,gesture_roll_number,84.541167,2
57,"{'z': 0.73645, 'stability': 'Stable', 'timesta...",device,0.0,85,7,-7.991331,9,76,-67.340333,75,10,85.0,89.411765,11.764706,gesture_roll_number,84.541167,2
58,"{'activity_confidence': 80, 'activity': 'Still...",device,0.0,100,8,179.464782,100,0,-77.994724,87,13,100.0,0.000000,13.000000,gesture_roll_number,84.541167,1
59,"{'activity': 'Still', 'stability': 'Stable', '...",device,0.0,96,9,-176.259097,100,4,-86.372440,96,0,96.0,4.166667,0.000000,gesture_roll_number,84.541167,1
60,"{'calibration_status': 2, 'duration': 231, 'x'...",device,0.0,46,10,-7.100933,8,38,-59.022820,66,20,46.0,82.608696,43.478261,gesture_roll_number,84.541167,2


In [12]:
df[df["experiment_type"] == "slider_number"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
86,"{'stability': 'In motion', 'activity_confidenc...",slider,91.702067,89,1,-7.474058,8,81,-36.446336,40,49,2.702067,91.011236,55.056180,slider_number,92.890343,2
87,"{'z': 0.847412, 'activity_confidence': 97, 'ti...",slider,6.660311,11,2,-7.474058,8,3,-36.446336,40,29,4.339689,27.272727,263.636364,slider_number,92.890343,2
88,"{'z': 0.847412, 'duration': 231, 'activity': '...",slider,63.674192,61,3,-7.474058,8,53,-36.446336,40,21,2.674192,86.885246,34.426230,slider_number,92.890343,2
89,"{'stability': 'In motion', 'z': 0.847412, 'tim...",slider,97.059559,94,4,-7.474058,8,86,-36.446336,40,54,3.059559,91.489362,57.446809,slider_number,92.890343,2
90,"{'z': 0.847412, 'x': -0.25531, 'w': -0.189819,...",slider,89.814848,79,5,-7.474058,8,71,-36.446336,40,39,10.814848,89.873418,49.367089,slider_number,92.890343,2
91,"{'calibration_status': 2, 'w': -0.189819, 'sta...",slider,15.061070,40,6,-7.474058,8,32,-36.446336,40,0,24.938930,80.000000,0.000000,slider_number,92.890343,2
92,"{'activity_confidence': 97, 'x': -0.25531, 'du...",slider,23.348454,34,7,-7.474058,8,26,-36.446336,40,6,10.651546,76.470588,17.647059,slider_number,92.890343,2
93,"{'x': -0.25531, 'duration': 231, 'y': -0.42511...",slider,50.490977,49,8,-7.474058,8,41,-36.446336,40,9,1.490977,83.673469,18.367347,slider_number,92.890343,2
94,"{'calibration_status': 2, 'timestamp': 1719320...",slider,11.629489,16,9,-7.474058,8,8,-36.446336,40,24,4.370511,50.000000,150.000000,slider_number,92.890343,2
95,"{'duration': 231, 'x': -0.25531, 'y': -0.42511...",slider,35.755673,39,10,-7.474058,8,31,-36.446336,40,1,3.244327,79.487179,2.564103,slider_number,92.890343,2


In [13]:
df[df["experiment_type"] == "slider_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,experiment_time_spent,calibration
71,"{'activity': 'Still', 'stability': 'In motion'...",slider,5.326400,21,1,-7.474058,4,17,-36.446336,20,1,15.673600,80.952381,4.761905,slider_greyscale,197.632769,2
72,"{'z': 0.847412, 'y': -0.42511, 'x': -0.25531, ...",slider,39.455783,39,3,-7.474058,4,35,-36.446336,20,19,0.455783,89.743590,48.717949,slider_greyscale,197.632769,2
73,"{'y': -0.42511, 'calibration_status': 2, 'z': ...",slider,40.328140,16,4,-7.474058,4,12,-36.446336,20,4,24.328140,75.000000,25.000000,slider_greyscale,197.632769,2
74,"{'stability': 'In motion', 'w': -0.189819, 'x'...",slider,22.779536,7,5,-7.474058,4,3,-36.446336,20,13,15.779536,42.857143,185.714286,slider_greyscale,197.632769,2
75,"{'w': -0.189819, 'duration': 231, 'x': -0.2553...",slider,1.320304,2,6,-7.474058,4,2,-36.446336,20,18,0.679696,100.000000,900.000000,slider_greyscale,197.632769,2
76,"{'y': -0.42511, 'w': -0.189819, 'activity': 'S...",slider,0.426171,1,7,-7.474058,4,3,-36.446336,20,19,0.573829,300.000000,1900.000000,slider_greyscale,197.632769,2
77,"{'calibration_status': 2, 'duration': 231, 'ti...",slider,0.000000,18,8,-7.474058,4,14,-36.446336,20,2,18.000000,77.777778,11.111111,slider_greyscale,197.632769,2
78,"{'y': -0.42511, 'activity': 'Still', 'stabilit...",slider,25.844795,31,9,-7.474058,4,27,-36.446336,20,11,5.155205,87.096774,35.483871,slider_greyscale,197.632769,2
79,"{'calibration_status': 2, 'w': -0.189819, 'x':...",slider,7.510580,15,11,-7.474058,4,11,-36.446336,20,5,7.489420,73.333333,33.333333,slider_greyscale,197.632769,2
80,"{'y': -0.42511, 'w': -0.189819, 'stability': '...",slider,48.807005,26,13,-7.474058,4,22,-36.446336,20,6,22.807005,84.615385,23.076923,slider_greyscale,197.632769,2


In [14]:
g_pitch_greyscale_df = df[(df["experiment_type"] == "gesture_pitch_greyscale")]
g_pitch_number_df = df[(df["experiment_type"] == "gesture_pitch_number")]
g_roll_greyscale_df = df[(df["experiment_type"] == "gesture_roll_greyscale")]
g_roll_number_df = df[(df["experiment_type"] == "gesture_roll_number")]
g_slider_greyscale_df = df[(df["experiment_type"] == "slider_greyscale")]
g_slider_number_df = df[(df["experiment_type"] == "slider_number")]

fig = sp.make_subplots(rows=6, cols=2, subplot_titles=(
    'Pitch - Number (Scaled 0-100)',
    'Pitch Error - Number (Scaled 0-100)',
    'Roll - Number (Scaled 0-100)',
    'Roll Error - Number (Scaled 0-100)',
    'Slider - Number (Scaled 0-100)',
    'Slider Error - Number (Scaled 0-100)',
    'Pitch - Greyscale (Scaled 0-49)',
    'Pitch Error - Greyscale (Scaled 0-49)',
    'Roll - Greyscale (Scaled 0-49)',
    'Roll Error - Greyscale (Scaled 0-49)',
    'Slider - Greyscale (Scaled 0-49)',
    'Slider Error - Greyscale (Scaled 0-49)'
))

def add_scatter_with_equal_axes(df, truth_info, error_y, scale, row, col, fig):
    legend_repeated = row < 2 and col < 2
    trace = px.scatter(df, x="truth", y=truth_info['truth'], error_y=error_y, color_discrete_sequence=[px.colors.qualitative.Plotly[row - 1]]).data[0]
    fig.add_trace(trace, row=row, col=col)
    fig.update_xaxes(tickmode='linear', dtick=10, row=row, col=col)
    fig.update_yaxes(tickmode='linear', dtick=10, row=row, col=col)
    fig.add_shape(
        type="line",
        x0=0, y0=0, x1=scale, y1=scale,
        line=dict(color="Black", width=1, dash="dot"),
        row=row, col=col,
        showlegend=legend_repeated,
        name="Linear Truth")

add_scatter_with_equal_axes(g_pitch_number_df, experiment_truth_type["gesture_pitch_number"], None, scale_type["gesture_pitch_number"], 1, 1, fig)
add_scatter_with_equal_axes(g_pitch_number_df, experiment_truth_type["gesture_pitch_number"], experiment_truth_type["gesture_pitch_number"]["truth_diff"], scale_type["gesture_pitch_number"], 1, 2, fig)

add_scatter_with_equal_axes(g_roll_number_df, experiment_truth_type["gesture_roll_number"], None, scale_type["gesture_roll_number"], 2, 1, fig)
add_scatter_with_equal_axes(g_roll_number_df, experiment_truth_type["gesture_roll_number"], experiment_truth_type["gesture_roll_number"]["truth_diff"], scale_type["gesture_roll_number"], 2, 2, fig)

add_scatter_with_equal_axes(g_slider_number_df, experiment_truth_type["slider_number"], None, scale_type["slider_number"], 3, 1, fig)
add_scatter_with_equal_axes(g_slider_number_df, experiment_truth_type["slider_number"], experiment_truth_type["slider_number"]["truth_diff"], scale_type["slider_number"], 3, 2, fig)

add_scatter_with_equal_axes(g_pitch_greyscale_df, experiment_truth_type["gesture_pitch_greyscale"], None, scale_type["gesture_pitch_greyscale"], 4, 1, fig)
add_scatter_with_equal_axes(g_pitch_greyscale_df, experiment_truth_type["gesture_pitch_greyscale"], experiment_truth_type["gesture_pitch_greyscale"]["truth_diff"], scale_type["gesture_pitch_greyscale"], 4, 2, fig)

add_scatter_with_equal_axes(g_roll_greyscale_df, experiment_truth_type["gesture_roll_greyscale"], None, scale_type["gesture_roll_greyscale"], 5, 1, fig)
add_scatter_with_equal_axes(g_roll_greyscale_df, experiment_truth_type["gesture_roll_greyscale"], experiment_truth_type["gesture_roll_greyscale"]["truth_diff"], scale_type["gesture_roll_greyscale"], 5, 2, fig)

add_scatter_with_equal_axes(g_slider_greyscale_df, experiment_truth_type["slider_greyscale"], None, scale_type["slider_greyscale"], 6, 1, fig)
add_scatter_with_equal_axes(g_slider_greyscale_df, experiment_truth_type["slider_greyscale"], experiment_truth_type["slider_greyscale"]["truth_diff"], scale_type["slider_greyscale"], 6, 2, fig)

fig.update_layout(height=6*400, width=2*400, title_text="Sensor Readings Scatter Plots")
fig.update_xaxes(title_text="Truth Values")
fig.update_yaxes(title_text="Sensor Readings")

fig['layout']['yaxis1'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis2'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis3'].update(title='Roll Sensor Readings')
fig['layout']['yaxis4'].update(title='Roll Sensor Readings')
fig['layout']['yaxis5'].update(title='Slider Readings')
fig['layout']['yaxis6'].update(title='Slider Readings')
fig['layout']['yaxis7'].update(title='Roll Sensor Readings')
fig['layout']['yaxis8'].update(title='Roll Sensor Readings')
fig['layout']['yaxis9'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis10'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis11'].update(title='Slider Readings')
fig['layout']['yaxis12'].update(title='Slider Readings')

fig.show()

In [15]:
def create_histogram_truth_plots(experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['inputType'] == truth_type["input_type"])]

    fig = px.histogram(filtered_df, x=truth_type["truth_diff"],
                    title=f'{truth_type['label']} Differences - {experiment_type}',
                    labels={f'{truth_type["truth_diff"]}': f'{truth_type["label"]} Difference'})
    fig.update_layout(width=500, height=500)
    fig.show()
    
for experiment_type, truth_type in experiment_truth_type.items():
    create_histogram_truth_plots(experiment_type, truth_type)

In [16]:
def calculate_summary_statistics(experiment_type, truth_type):
    filtered_df = df[df['experiment_type'] == experiment_type]
    grouped = filtered_df.groupby('truth')[truth_type].agg(['mean', 'std', 'min', 'max', 'count', 'median'])
    grouped['iqr'] = filtered_df.groupby('truth')[truth_type].apply(iqr)
    grouped['skew'] = filtered_df.groupby('truth')[truth_type].apply(skew)
    grouped['kurtosis'] = filtered_df.groupby('truth')[truth_type].apply(kurtosis)
    
    # Kurtosis and skew calculate NaNs on certain truths (0 and 100), but it's not an issue. Filling NaNs with zeroes
    grouped = grouped.fillna(0)
    return grouped.reset_index()

def generate_summary_tables():
    summary_tables = {}
    for experiment_type, truth_info in experiment_truth_type.items():
        summary_table = calculate_summary_statistics(experiment_type, truth_info['truth'])
        summary_table['experiment_type'] = experiment_type
        summary_tables[experiment_type] = summary_table
    return summary_tables

def calculate_truth_diff_percentages(experiment_type, truth_diff_column):
    filtered_df = df[df['experiment_type'] == experiment_type]
    count = filtered_df.shape[0]
    within_5 = (filtered_df[truth_diff_column].abs() <= 5).sum() / count * 100
    within_10 = (filtered_df[truth_diff_column].abs() <= 10).sum() / count * 100
    within_15 = (filtered_df[truth_diff_column].abs() <= 15).sum() / count * 100
    within_20 = (filtered_df[truth_diff_column].abs() <= 20).sum() / count * 100
    within_25 = (filtered_df[truth_diff_column].abs() <= 25).sum() / count * 100
    return within_5, within_10, within_15, within_20, within_25

def calculate_averages_per_experiment_type(summary_tables):
    averages = []
    for experiment_type, summary_table in summary_tables.items():
        within_5, within_10, within_15, within_20, within_25 = calculate_truth_diff_percentages(experiment_type, experiment_truth_type[experiment_type]['truth_diff'])
        
        time_spent_df = df[df['experiment_type'] == experiment_type].drop_duplicates(subset=['experiment_type'])
        time_spent = time_spent_df['experiment_time_spent'].abs().mean()
        
        averages.append({
            'experiment_type': experiment_type,
            'time_spent': time_spent,
            'mean': summary_table['mean'].mean(),
            'std': summary_table['std'].mean(),
            'min': summary_table['min'].min(),
            'max': summary_table['max'].max(),
            'count': summary_table['count'].count(),
            'median': summary_table['median'].median(),
            'iqr': summary_table['iqr'].mean(),
            'skew': summary_table['skew'].mean(),
            'kurtosis': summary_table['kurtosis'].mean(),
            'within_5_percent': within_5,
            'within_10_percent': within_10,
            'within_15_percent': within_15,
            'within_20_percent': within_20,
            'within_25_percent': within_25
        })
    return pd.DataFrame(averages)

summary_tables = generate_summary_tables()
averages_df = calculate_averages_per_experiment_type(summary_tables)

display(averages_df)

for _, summary_table in summary_tables.items():
    display(summary_table)

,experiment_type,time_spent,mean,std,min,max,count,median,iqr,skew,kurtosis,within_5_percent,within_10_percent,within_15_percent,within_20_percent,within_25_percent
0,gesture_pitch_number,112.335365,52.500000,0.000000,12.000000,79.000000,20,54.000000,0.000000,0.000000e+00,0.000000,35.000000,65.000000,100.000000,100.000000,100.0
1,gesture_pitch_greyscale,139.322510,26.615385,0.000000,2.000000,49.000000,13,25.000000,0.000000,0.000000e+00,0.000000,15.384615,38.461538,61.538462,100.000000,100.0
2,gesture_roll_number,84.541167,66.000000,0.000000,37.000000,96.000000,20,70.500000,0.000000,0.000000e+00,0.000000,40.000000,45.000000,65.000000,90.000000,95.0
3,gesture_roll_greyscale,206.524229,25.166667,0.000000,12.000000,41.000000,18,24.500000,0.000000,0.000000e+00,0.000000,5.555556,61.111111,77.777778,100.000000,100.0
4,slider_number,92.890343,50.607768,0.000000,5.583582,97.059559,20,51.939619,0.000000,0.000000e+00,0.000000,80.000000,85.000000,95.000000,95.000000,100.0
5,slider_greyscale,197.632769,19.029431,1.002325,0.000000,48.807005,14,17.011175,0.708751,2.078058e-17,-0.142857,20.000000,40.000000,46.666667,73.333333,100.0


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,4,12.0,0.0,12,12,1,12.0,0.0,0.0,0.0,gesture_pitch_number
1,17,27.0,0.0,27,27,1,27.0,0.0,0.0,0.0,gesture_pitch_number
2,28,30.0,0.0,30,30,1,30.0,0.0,0.0,0.0,gesture_pitch_number
3,35,38.0,0.0,38,38,1,38.0,0.0,0.0,0.0,gesture_pitch_number
4,36,39.0,0.0,39,39,1,39.0,0.0,0.0,0.0,gesture_pitch_number
5,44,46.0,0.0,46,46,1,46.0,0.0,0.0,0.0,gesture_pitch_number
6,50,54.0,0.0,54,54,1,54.0,0.0,0.0,0.0,gesture_pitch_number
7,53,55.0,0.0,55,55,1,55.0,0.0,0.0,0.0,gesture_pitch_number
8,56,50.0,0.0,50,50,1,50.0,0.0,0.0,0.0,gesture_pitch_number
9,57,52.0,0.0,52,52,1,52.0,0.0,0.0,0.0,gesture_pitch_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,16.0,0.0,16,16,1,16.0,0.0,0.0,0.0,gesture_pitch_greyscale
1,5,12.0,0.0,12,12,1,12.0,0.0,0.0,0.0,gesture_pitch_greyscale
2,12,25.0,0.0,25,25,1,25.0,0.0,0.0,0.0,gesture_pitch_greyscale
3,20,7.0,0.0,7,7,1,7.0,0.0,0.0,0.0,gesture_pitch_greyscale
4,22,2.0,0.0,2,2,1,2.0,0.0,0.0,0.0,gesture_pitch_greyscale
5,26,30.0,0.0,30,30,1,30.0,0.0,0.0,0.0,gesture_pitch_greyscale
6,29,45.0,0.0,45,45,1,45.0,0.0,0.0,0.0,gesture_pitch_greyscale
7,30,38.0,0.0,38,38,1,38.0,0.0,0.0,0.0,gesture_pitch_greyscale
8,35,49.0,0.0,49,49,1,49.0,0.0,0.0,0.0,gesture_pitch_greyscale
9,37,20.0,0.0,20,20,1,20.0,0.0,0.0,0.0,gesture_pitch_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,9,41.0,0.0,41,41,1,41.0,0.0,0.0,0.0,gesture_roll_number
1,20,37.0,0.0,37,37,1,37.0,0.0,0.0,0.0,gesture_roll_number
2,23,40.0,0.0,40,40,1,40.0,0.0,0.0,0.0,gesture_roll_number
3,26,40.0,0.0,40,40,1,40.0,0.0,0.0,0.0,gesture_roll_number
4,33,48.0,0.0,48,48,1,48.0,0.0,0.0,0.0,gesture_roll_number
5,37,54.0,0.0,54,54,1,54.0,0.0,0.0,0.0,gesture_roll_number
6,46,66.0,0.0,66,66,1,66.0,0.0,0.0,0.0,gesture_roll_number
7,54,69.0,0.0,69,69,1,69.0,0.0,0.0,0.0,gesture_roll_number
8,58,81.0,0.0,81,81,1,81.0,0.0,0.0,0.0,gesture_roll_number
9,60,77.0,0.0,77,77,1,77.0,0.0,0.0,0.0,gesture_roll_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,6,14.0,0.0,14,14,1,14.0,0.0,0.0,0.0,gesture_roll_greyscale
1,9,28.0,0.0,28,28,1,28.0,0.0,0.0,0.0,gesture_roll_greyscale
2,13,12.0,0.0,12,12,1,12.0,0.0,0.0,0.0,gesture_roll_greyscale
3,15,23.0,0.0,23,23,1,23.0,0.0,0.0,0.0,gesture_roll_greyscale
4,17,29.0,0.0,29,29,1,29.0,0.0,0.0,0.0,gesture_roll_greyscale
5,19,29.0,0.0,29,29,1,29.0,0.0,0.0,0.0,gesture_roll_greyscale
6,23,41.0,0.0,41,41,1,41.0,0.0,0.0,0.0,gesture_roll_greyscale
7,24,18.0,0.0,18,18,1,18.0,0.0,0.0,0.0,gesture_roll_greyscale
8,25,16.0,0.0,16,16,1,16.0,0.0,0.0,0.0,gesture_roll_greyscale
9,27,20.0,0.0,20,20,1,20.0,0.0,0.0,0.0,gesture_roll_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,5,5.583582,0.0,5.583582,5.583582,1,5.583582,0.0,0.0,0.0,slider_number
1,10,9.044641,0.0,9.044641,9.044641,1,9.044641,0.0,0.0,0.0,slider_number
2,11,6.660311,0.0,6.660311,6.660311,1,6.660311,0.0,0.0,0.0,slider_number
3,16,11.629489,0.0,11.629489,11.629489,1,11.629489,0.0,0.0,0.0,slider_number
4,27,26.547758,0.0,26.547758,26.547758,1,26.547758,0.0,0.0,0.0,slider_number
5,31,22.167535,0.0,22.167535,22.167535,1,22.167535,0.0,0.0,0.0,slider_number
6,34,23.348454,0.0,23.348454,23.348454,1,23.348454,0.0,0.0,0.0,slider_number
7,39,35.755673,0.0,35.755673,35.755673,1,35.755673,0.0,0.0,0.0,slider_number
8,40,15.061070,0.0,15.061070,15.061070,1,15.061070,0.0,0.0,0.0,slider_number
9,49,50.490977,0.0,50.490977,50.490977,1,50.490977,0.0,0.0,0.0,slider_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,1,0.426171,0.000000,0.426171,0.426171,1,0.426171,0.00000,0.000000e+00,0.0,slider_greyscale
1,2,11.242814,14.032548,1.320304,21.165323,2,11.242814,9.92251,2.909281e-16,-2.0,slider_greyscale
2,6,30.175503,0.000000,30.175503,30.175503,1,30.175503,0.00000,0.000000e+00,0.0,slider_greyscale
3,7,22.779536,0.000000,22.779536,22.779536,1,22.779536,0.00000,0.000000e+00,0.0,slider_greyscale
4,13,2.565144,0.000000,2.565144,2.565144,1,2.565144,0.00000,0.000000e+00,0.0,slider_greyscale
5,15,7.510580,0.000000,7.510580,7.510580,1,7.510580,0.00000,0.000000e+00,0.0,slider_greyscale
6,16,40.328140,0.000000,40.328140,40.328140,1,40.328140,0.00000,0.000000e+00,0.0,slider_greyscale
7,18,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.00000,0.000000e+00,0.0,slider_greyscale
8,20,25.841558,0.000000,25.841558,25.841558,1,25.841558,0.00000,0.000000e+00,0.0,slider_greyscale
9,21,5.326400,0.000000,5.326400,5.326400,1,5.326400,0.00000,0.000000e+00,0.0,slider_greyscale


In [17]:
for experiment_type, truth_info in experiment_truth_type.items():
    filtered_df = df[df['experiment_type'] == experiment_type]
    truth_column = truth_info['truth']
    response_column = truth_info['truth_diff']

    pearson_corr, _ = pearsonr(filtered_df[truth_column], filtered_df[response_column])
    print(f"Pearson correlation coefficient for {experiment_type}: {pearson_corr}")

    t_stat, p_value = ttest_ind(filtered_df[truth_column], filtered_df[response_column])
    print(f"t-statistic for {experiment_type}: {t_stat}, p-value: {p_value}\n")

Pearson correlation coefficient for gesture_pitch_number: 0.4764919937374372
t-statistic for gesture_pitch_number: 11.039091378439553, p-value: 2.0396569652974474e-13

Pearson correlation coefficient for gesture_pitch_greyscale: -0.4656433595482267
t-statistic for gesture_pitch_greyscale: 3.155820927652234, p-value: 0.00427336639289631

Pearson correlation coefficient for gesture_roll_number: -0.525108985726155
t-statistic for gesture_roll_number: 12.658642649843497, p-value: 3.3270862870836085e-15

Pearson correlation coefficient for gesture_roll_greyscale: 0.4235509593160956
t-statistic for gesture_roll_greyscale: 6.064190631696433, p-value: 7.110006797202137e-07

Pearson correlation coefficient for slider_number: -0.22779938183322582
t-statistic for slider_number: 5.969863224368625, p-value: 6.261335567354586e-07

Pearson correlation coefficient for slider_greyscale: 0.3067524467967775
t-statistic for slider_greyscale: 1.1772816905921604, p-value: 0.24899379611516897

